# Programming Fortran With GNU Fortran

* %cd Change the current working directory to `.\demo\.

In [1]:
%cd demo

F:\SEU\SEE\PySEE\home\notebook\demo


In [3]:
%pwd

'F:\\SEU\\SEE\\PySEE\\home\\notebook\\demo'

## 1. gfortran - the GNU Fortran compiler

https://gcc.gnu.org/wiki/GFortran

Gfortran is the name of the GNU Fortran project, developing a free Fortran 95/2003/2008 compiler for GCC, the GNU Compiler Collection. The gfortran development effort uses an open development environment in order to attract a larger team of developers and to ensure that gfortran can work on multiple architectures and diverse environments. 



#### Debian/Ubuntu

In [ ]:
!sudo apt install gfortran

#### Windows

you have gfortran after install **MinGW-GW64**

you may verify the Gfortran installation by listing the version of Gfortran: 

In [2]:
!gfortran --version

GNU Fortran (MinGW-W64 x86_64-posix-dwarf, built by Brecht Sanders) 10.0.1 20200223 (experimental)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



## 2  Getting Started

Compile/Link a Simple Fortran Program - hello.f90

> **NOTE**: .f90 for **free form source Fortran**

In [3]:
%%file ./gfortran/hello.f90

program hello
  write(*,*) 'Fortran says Hello, world!'
end program hello

Overwriting ./gfortran/hello.f90


Then,use **gfortran** to the compile Fortran program

In [4]:
!gfortran -c -o ./obj/hello.o ./gfortran/hello.f90
!gfortran -o ./bin/hello ./obj/hello.o

In [6]:
!.\bin\hello

 Fortran says Hello, world!


## 3 Procedures

`Subroutines` and `functions` are called procedures

### 3.1 Subroutines

* use the `call` statement, `call <sub name>(args...)`

The subroutine is defined somewhere else:

```
subroutine <name>(args)
use ...
implicit none
variable declarations
executable statements
end subroutine <name>
```
**Dummy Arguments**

* Variable names in a procedure, only exist in that procedure

* Fortran is `pass-by-reference`

### 3.2 Functions

Purpose is to return **only one** result

The **function name** 

* defines the variable in the function

* is returned at the end of a function


### 3.3 Example Procedures

The `bisection method` is a root-finding method that applies to any continuous functions for which one knows two values with opposite signs. The method consists of repeatedly bisecting the interval defined by these values and then selecting the subinterval in which the function changes sign, and therefore must contain a root. 


In [5]:
%%file ./gfortran/bisect.f08
subroutine bisect(fun,xl,xr,x,f,tol)

implicit none

  ! arguments
  real(8),intent(inout) :: xl       ! left bound
  real(8),intent(inout) :: xr       ! right bound
  real(8),intent(inout) :: x        ! result
  real(8),intent(out)   :: f        ! residual
  real(8),intent(in)    :: tol      ! residual tolerance

  ! function argument
  real(8) :: fun
  external fun

  ! local varaibles
  real(8) :: fl     ! residual for left  bound
  real(8) :: fr     ! resdiual for right bound
  integer :: i      ! loop counter

  ! determine residual bounds
  fl = fun(xl)
  fr = fun(xr)

  ! begin loop
  do i = 1,100

    ! get midpoint
    x = 0.5_8*(xl + xr)

    ! evaluate resdiual at midpoint
    f = fun(x)

    ! check for convergence
    if (abs(f) < tol) exit

    ! reset the bounds
    if (f*fl < dble(0.0)) then

      ! move right bound info to mid
      xr = x
      fr = f

    else

      ! move left bound info to mid
      xl = x
      fl = f

    end if


    ! print out information
    print *,'Iteration:',i,' Residual:',f

  end do

end subroutine bisect


Overwriting ./gfortran/bisect.f08


### 3.4 Example Function

In [6]:
%%file ./gfortran/myfun.f08
function myfun(x)

  implicit none

  ! formal variables
  real(8) :: myfun   ! the function declaration, residual
  real(8) :: x       ! the independent variable

  myfun = x**2 - 4

end function myfun

Overwriting ./gfortran/myfun.f08


### 3.5 Main demo

In [7]:
%%file ./gfortran/findroot.f08
program findroot

  implicit none

  real(8) :: x   ! solution variable
  real(8) :: xl  ! lower bound
  real(8) :: xr  ! upper bound
  real(8) :: f   ! final residual
  real(8) :: tol ! tolerance

  ! function declaration
  external myfun

  ! ask user for guess
  x=1.2

  ! ask for bounds
  xl=0.1
  xr=2.0

  ! ask for tolerance
  tol=0.01

  ! solve by bisection
  call bisect(myfun,xl,xr,x,f,tol)

  ! print result
  print *,'Solution is:',x,' with final residual:',f

  ! terminate the program
  stop

end program findroot


Overwriting ./gfortran/findroot.f08


In [8]:
!gfortran -o ./obj/bisect.o -c ./gfortran/bisect.f08  
!gfortran -o ./obj/myfun.o -c ./gfortran/myfun.f08 
!gfortran -o ./obj/findroot.o -c ./gfortran/findroot.f08 
!gfortran -o ./bin/findroot.exe ./obj/findroot.o  ./obj/bisect.o  ./obj/myfun.o 

In [9]:
!.\bin\findroot.exe

 Iteration:           1  Residual:  -2.8974999984353778     
 Iteration:           2  Residual:  -1.6743749988637866     
 Iteration:           3  Residual: -0.89359374934341762     
 Iteration:           4  Residual: -0.46089843714958967     
 Iteration:           5  Residual: -0.23397460919426516     
 Iteration:           6  Residual: -0.11786865225200005     
 Iteration:           7  Residual:  -5.9154663039716837E-002
 Iteration:           8  Residual:  -2.9632415748287677E-002
 Iteration:           9  Residual:  -1.4829978931251375E-002
 Solution is:   1.9981445312514552       with final residual:  -7.4184322299024608E-003


## 4 Modules

Modules fulfill multiple purposes

Used for shared declarations (similar to headers)

Used for defining global data

Used for defining procedure interfaces

Think of them as a high-level interface in your code

Structure of a Module

```fortran
module <name>
use definitions
implicit none
static data definitions, global to the module
contains
procedure definitions and interfaces
end module <name>
```
**Example module class_Circle**


In [11]:
%%file ./gfortran/class_Circle.f08
module class_Circle

  implicit none
  private
  public :: circle_print

  real :: pi = 3.1415926535897931D0 ! module-wide private constant

  type, public :: Circle

    real :: radius ! the radius of a circle
    real :: area   ! the area of a circle
    real :: circum ! the circumference of a circle

  end type Circle

contains

!==============================================================================
! CIRCLE_AREA calculates the area of a circle
!==============================================================================

  function circle_area(this) result(area)

    type(Circle), intent(in)  :: this ! circle instance
    real                      :: area ! the area of the circle

    ! calculate the area
    area = pi * this%radius**2

  end function circle_area

!==============================================================================
! CIRCLE_CIRCUM calculate the circumference of a circle
!==============================================================================

  function circle_circum(this) result(circum)

    type(Circle), intent(in) :: this   ! circle instance
    real                     :: circum ! the circumference

    ! calculate circumference
    circum = 2*pi*this%radius

  end function circle_circum

!==============================================================================
! CIRCLE_PRINT prints information about the circle
!==============================================================================

  subroutine circle_print(this)

    type(Circle), intent(inout) :: this ! circle instance

    ! calculate area
    this%area = circle_area(this)

    ! calculate circumference
    this%circum = circle_circum(this)

    ! print results
    write(*,'("The area is:",T30,F0.4,/,"The circumference is:",T30,F0.4)')    &
   &           this%area,this%circum

  end subroutine circle_print

end module class_Circle


Overwriting ./gfortran/class_Circle.f08


In [12]:
%%file ./gfortran/circle_test.f08
program circle_test

  use class_Circle, only: Circle,circle_print

  implicit none

  type(Circle) :: acircle  ! a circle instance

  acircle%radius=3.2
 
  write(*,'("The radius is:",T30,F0.4)') acircle%radius
  ! print out results
  call circle_print(acircle)

  ! terminate the program
  stop

end program circle_test


Overwriting ./gfortran/circle_test.f08


In [13]:
%%file makefile-gfortran-circle.mk

SRC= ./gfortran/
BIN= ./bin/
OBJ= ./obj/

all: circle_test

circle_test: circleobj
	 gfortran -o $(BIN)circle_test.exe  $(OBJ)circle_test.o $(OBJ)class_Circle.mod
    
circleobj:
	 gfortran -o $(OBJ)class_Circle.mod -c  $(SRC)class_Circle.f08    
	 gfortran -o $(OBJ)circle_test.o  -c  $(SRC)circle_test.f08     


Overwriting makefile-gfortran-circle.mk


In [14]:
!make -f makefile-gfortran-circle.mk

gfortran -o ./obj/class_Circle.mod -c  ./gfortran/class_Circle.f08    
gfortran -o ./obj/circle_test.o  -c  ./gfortran/circle_test.f08     
gfortran -o ./bin/circle_test.exe  ./obj/circle_test.o ./obj/class_Circle.mod


In [15]:
!.\bin\circle_test

The radius is:               3.2000
The area is:                 32.1699
The circumference is:        20.1062


## Reference

1. MIT 22.901 Fortran Class https://github.com/bhermanmit/Fortran

2. Awesome list of Fortran libs https://github.com/rabbiabram/awesome-fortran

3. SEUIF97-API Fortran： https://github.com/PySEE/SEUIF97/tree/master/demo-fortran